Occasionally, someone stops by asking for help with restoring antique clocks. How
much should David charge per hour for mechanical repairs and how much should
LaDeana charge per hour for wood restoration assuming that they do not wish to
add more working hours and also do not wish to reduce company profit if one of them
is taking on a repair job?

David does not need to charge anything per hour for mechanical repairs since he is not the bottleneck (obviously he should charge something), but company profit is not reduced if he takes a repair job as long as the amount of time taken for repair jobs is not too large

LaDeana should charge at least $ 25.000 / hr

Problem 2:

In [8]:
#following Petrat
from pyomo.environ import *
from pyomo.opt import *
opt = solvers.SolverFactory("glpk")

model = ConcreteModel()
BIGNUM = 2**40

A = [1,2,3]# destinations
R = ['P1', 'P2', 'P3' , 'P4', 'P5'] # distribution centers
w={ (1, 'P1'): 41, (1, 'P2'): 39, (1, 'P3'): 42, (1, 'P4'): 38, (1, 'P5'): 39,
    (2, 'P1'): 55,(2, 'P2'): 51, (2, 'P3'): 56, (2, 'P4'): 52,(2, 'P5'): 53 ,
    (3, 'P1'): 48, (3, 'P2'): 45, (3, 'P3'): 50, (3, 'P4'): BIGNUM, (3, 'P5'): BIGNUM,
}# constraint matrix
d = { 1:700, 2: 1000, 3: 900} #demand constraints

s = {'P1':400,'P2':600,'P3':400,'P4':600, 'P5':1000} # demand constraints

def supply_rule(model, j):
    return sum(model.x[i,j] for i in A) <= s[j]

def demand_rule(model, i):
    return sum(model.x[i,j] for j in R) == d[i]


model.x = Var(A,R, within=NonNegativeReals)

model.z = Objective(expr=sum( (    w[i,j] * model.x[i,j] ) for j in R for i in A), sense=minimize)
#set constraints
model.supply = Constraint(R, rule=supply_rule)
model.demand=Constraint(A, rule=demand_rule)



model.dual = Suffix(direction=Suffix.IMPORT)
results = opt.solve(model)

print(model.x.get_values())
print(f"total cost of transportation: {model.z.expr()}")


{(1, 'P1'): 0.0, (1, 'P2'): 0.0, (1, 'P3'): 0.0, (1, 'P4'): 0.0, (1, 'P5'): 700.0, (2, 'P1'): 0.0, (2, 'P2'): 400.0, (2, 'P3'): 0.0, (2, 'P4'): 600.0, (2, 'P5'): 0.0, (3, 'P1'): 400.0, (3, 'P2'): 200.0, (3, 'P3'): 300.0, (3, 'P4'): 0.0, (3, 'P5'): 0.0}
total cost of transportation: 122100.0


(Based on HL, Problem 8.3-2.) Four cargo ships will be used for shipping goods from
one port to four other ports (labeled 1, 2, 3, 4). Any ship can be used for making any
one of these four trips. However, because of differences in the ships and cargoes, the
total cost of loading, transporting, and unloading the goods for the different ship-port
combinations varies considerably, as shown in the following table

The objective is to assign the four ships to four different ports in such a way as to minimize
the total cost for all four shipments.


In [16]:

model = ConcreteModel()
BIGNUM = 2**40

PORTS = [1,2,3,4]# destinations
SHIPS= ['S1', 'S2', 'S3', 'S4']

w={ (1, 'S1'): 500, (1, 'S2'): 600, (1, 'S3'): 700, (1, 'S4'): 500, 
    (2, 'S1'): 400, (2, 'S2'): 600, (2, 'S3'): 500, (2, 'S4'): 400,
    (3, 'S1'): 600, (3, 'S2'): 700, (3, 'S3'): 700, (3, 'S4'): 600, 
    (4, 'S1'): 700, (4, 'S2'): 500, (4, 'S3'): 600, (4, 'S4'): 600, 
}# constraint matrix
# d = { 1:700, 2: 1000, 3: 900} #demand constraints

# s = {'P1':400,'P2':600,'P3':400,'P4':600, 'P5':1000} # demand constraints

# def supply_rule(model, j):
#     return sum(model.x[i,j] for i in A) <= s[j]

# def demand_rule(model, i):
#     return sum(model.x[i,j] for j in R) == d[i]


model.x = Var(PORTS,SHIPS, within=Boolean)

model.z = Objective(expr=sum( (    w[i,j] * model.x[i,j] ) for j in SHIPS for i in PORTS), sense=minimize)
#all ports must be visited by at least one ship
def all_ports_visited(model, port_num):
    return sum( [ model.x[port_num, ship] for ship in SHIPS ]  ) >= 1

def each_ship_one_trip(model, ship):
    return sum( [model.x[port, ship] for port in PORTS ]) == 1
model.demand=Constraint(PORTS, rule=all_ports_visited)
model.supply=Constraint(SHIPS, rule=each_ship_one_trip)


# model.dual = Suffix(direction=Suffix.IMPORT)
results = opt.solve(model)

print(model.x.get_values())
print(f"total cost of transportation: {model.z.expr()}")


{(1, 'S1'): 0.0, (1, 'S2'): 0.0, (1, 'S3'): 0.0, (1, 'S4'): 1.0, (2, 'S1'): 0.0, (2, 'S2'): 0.0, (2, 'S3'): 1.0, (2, 'S4'): 0.0, (3, 'S1'): 1.0, (3, 'S2'): 0.0, (3, 'S3'): 0.0, (3, 'S4'): 0.0, (4, 'S1'): 0.0, (4, 'S2'): 1.0, (4, 'S3'): 0.0, (4, 'S4'): 0.0}
total cost of transportation: 2100.0


In [ ]:
#now we change what we had before . . .. 
model = ConcreteModel()
BIGNUM = 2**40

PORTS = [1,2,3,4]# destinations
SHIPS= ['S1', 'S2', 'S3', 'S4']

w={ (1, 'S1'): 500, (1, 'S2'): 600, (1, 'S3'): 700, (1, 'S4'): 500, 
    (2, 'S1'): 400, (2, 'S2'): 600, (2, 'S3'): 500, (2, 'S4'): 400,
    (3, 'S1'): 600, (3, 'S2'): 700, (3, 'S3'): 700, (3, 'S4'): 600, 
    (4, 'S1'): 700, (4, 'S2'): 500, (4, 'S3'): 600, (4, 'S4'): 600, 
}# constraint matrix
# d = { 1:700, 2: 1000, 3: 900} #demand constraints

# s = {'P1':400,'P2':600,'P3':400,'P4':600, 'P5':1000} # demand constraints

# def supply_rule(model, j):
#     return sum(model.x[i,j] for i in A) <= s[j]

# def demand_rule(model, i):
#     return sum(model.x[i,j] for j in R) == d[i]


model.x = Var(PORTS,SHIPS, within=NonNegativeReals)

model.z = Objective(expr=sum( (    w[i,j] * model.x[i,j] ) for j in SHIPS for i in PORTS), sense=minimize)
#all ports must be visited by at least one ship
def all_ports_visited(model, port_num):
    return sum( [ model.x[port_num, ship] for ship in SHIPS ]  ) >= 1

def each_ship_one_trip(model, ship):
    return sum( [model.x[port, ship] for port in PORTS ]) == 1
model.demand=Constraint(PORTS, rule=all_ports_visited)
model.supply=Constraint(SHIPS, rule=each_ship_one_trip)


# model.dual = Suffix(direction=Suffix.IMPORT)
results = opt.solve(model)

print(model.x.get_values())
print(f"total cost of transportation: {model.z.expr()}")
